# Exercise - SABR Calibration and Hedging


#### Notation Commands

$$\newcommand{\done}{d_{1}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\vega}{\nu}$$


## Data

Use `../data/skew_data.xlsx`, which contains futures and options data for various assets.

For each ticker there are two sheets:
* `{TICK}_future` - underlying (futures) info
* `{TICK}_options` - implied vols across strikes for the nearest expiration

Throughout this exercise, use **`SFRA`** (3-Month SOFR futures).

Feel free to use the `sabr` and `sabrATM` functions in `cmds/volskew.py`.


# 1. Fit SABR to Market Data


## 1.1.

Load the implied-volatility data for SFRA.

Create a scatter plot of implied volatility vs. strike price.
What shape does the volatility smile/skew take for this asset?


## 1.2.

Fit a SABR curve to the quoted implied volatilities.

Recall that we typically set $\beta$ rather than directly estimating it.

Plot the fitted SABR curves corresponding to:
* $\beta=0$ (normal model)
* $\beta=0.5$ (CIR-style)
* $\beta=1$ (lognormal model)

Which value of $\beta$ provides the best visual fit to the market data?

Continuing, we'll use the $\beta=0.5$ curve.


## 1.3.

For the $\beta=0.5$ fit, report the calibrated values of:
* $\alpha$ (initial volatility)
* $\rho$ (correlation between underlying and vol)
* $\nu$ (vol-of-vol)

Interpret these parameter values.
What does each tell you about the volatility dynamics of SOFR futures?


# 2. Parameter Sensitivity


## 2.1.

Holding $\rho$ and $\nu$ fixed at their calibrated values, vary $\alpha$ by $\pm 20\%$.

Plot the resulting SABR curves. How does $\alpha$ affect:
* The level of the volatility curve?
* The shape of the skew?


## 2.2.

Now hold $\alpha$ and $\nu$ fixed, and vary $\rho$ across the range $[-0.8,\; -0.4,\; 0,\; 0.4,\; 0.8]$.

Plot the resulting SABR curves.
How does $\rho$ shift the skew?
Why does a negative $\rho$ produce a downward-sloping skew?


## 2.3.

Finally, hold $\alpha$ and $\rho$ fixed, and vary $\nu$ across $[0.2,\; 0.4,\; 0.6,\; 0.8,\; 1.0]$.

Plot the resulting SABR curves. How does $\nu$ (vol-of-vol) affect:
* The curvature of the smile?
* The wings of the distribution?


# 3. Calibration Diagnostics


## 3.1.

Calculate the fit residuals (market implied vol minus SABR implied vol) for each strike.

Report the root-mean-square error (RMSE) for your $\beta=0.5$ fit.


## 3.2.

Create a residual plot showing the calibration error at each strike.

Where does SABR fit poorly? (Deep OTM puts? Deep OTM calls?)
What might explain these systematic errors?


## 3.3.

Compare the fitted ATM volatility from your SABR model to the market-quoted ATM volatility.

How close is the match?
Why is it important for the model to match ATM vol accurately?


# 4. Skew-Adjusted Delta


## 4.1.

Consider the ATM call option on SFRA futures.

Report the delta of the position using Black's formula for futures.

#### Note
For Black's formula on futures, the delta is

$$\Delta_F = Z(t,T)\;\normcdf(\done)$$

where $Z(t,T)$ is the discount factor to option expiration, and

$$\done = \frac{\ln\!\left(\frac{F}{K}\right) + \frac{1}{2}\sigma^2\tau}{\sigma\sqrt{\tau}}$$


## 4.2.

Suppose the futures price **decreases** by `0.50` points (corresponding to a 50bp increase in the underlying SOFR rate).

Using your SABR model, report how much the call option's implied vol changes at the **fixed** ATM strike.
(The strike that was ATM is now OTM.)

Using this, numerically calculate the derivative

$$\frac{\partial\sigma}{\partial F}$$


## 4.3.

For the 50bp shock, what is the **skew-adjusted delta**?

$$\Delta_{\text{adjusted}} = \Delta + \vega \cdot \frac{\partial\sigma}{\partial F}$$

How much does it differ from the classic (naive) delta?


## 4.4.

Re-do the calculations for an OTM call.
Choose a strike substantially OTM.


# 5. Vol Path and Decomposition


In Section 4, we found that the implied vol at a **fixed strike** changes substantially when the underlying moves, producing a large skew-adjusted delta correction.

But how much of that correction comes from the vol *level* changing, vs. simply sliding along a rigid skew curve?

## 5.1.

Compute the **vol path**: the SABR-implied ATM vol as the underlying moves.

For a grid of hypothetical underlying values spanning the strike range, calculate the ATM vol at each point (i.e., set strike = underlying). Plot the result.

This vol path traces the "backbone" of the SABR model: $\sigma_{\text{ATM}} \approx \alpha / F^{1-\beta}$.


## 5.2.

Using the vol path, compute $\frac{\partial \sigma_{\text{ATM}}}{\partial F}$ for the same 50bp shock as Section 4.

Decompose the total fixed-strike vol change into two channels:

* **Vol-level** (backbone): how much ATM vol itself changes, $\frac{\partial \sigma_{\text{ATM}}}{\partial F}$.

* **Moneyness** (residual): the remainder,

$$\left.\frac{\partial \sigma}{\partial F}\right|_{\text{fixed }K} \;-\; \frac{\partial \sigma_{\text{ATM}}}{\partial F}$$

Report the share of the total delta correction attributable to each channel.


## 5.3.

The decomposition above is specific to the SABR model with stable parameters.

Under what market conditions would you expect the **vol-level channel** to dominate instead?


# 6. Cross-Asset Comparison (Optional)


## 6.1.

Fit SABR (with $\beta=0.5$) to **`TYA`** (10-Year US Treasury Note Futures).

Create a table comparing the calibrated parameters ($\alpha$, $\rho$, $\nu$) for SFRA and TYA.


## 6.2.

Which asset shows the stronger skew (more negative $\rho$)?
Which shows more smile curvature (higher $\nu$)?

Do these differences align with your intuition about these markets?
